In [178]:
# problem 1
lines = sc.textFile("input/*")
words = lines.flatMap(lambda l: l.split(" "))
results = words.map(lambda word: (word, 1)).reduceByKey(lambda word, count: word + count).collect()
for (word, count) in results:
    print "%-10s : %i" % (word, count)

Goodbye    : 1
World      : 2
Bye        : 1
Hello      : 2
Hadoop     : 2


In [179]:
# problem 2 - create table
from pyspark.sql import SQLContext
from pyspark.sql.types import *

sqlContext = SQLContext(sc)

lines = sc.textFile("purchases.txt")
purchases = lines.map(lambda l: [float(value) if index == 3 else value for index, value in enumerate(l.split(','))])

fields = [StructField("timestamp", StringType()), StructField("location", StringType()), 
         StructField("category", StringType()), StructField("price", FloatType()),
         StructField("card", StringType())]

schema = StructType(fields)

schemaPurchases = sqlContext.createDataFrame(purchases, schema)
schemaPurchases.registerTempTable("purchases")

In [180]:
# problem 2 - query 1
query1 = "SELECT AVG(price) AS avg_price FROM purchases WHERE card='MasterCard'"
results = sqlContext.sql(query1).map(lambda l: l.avg_price)
for result in results.collect():
    print result


275.067731742


In [177]:
# problem 2 - query 2
query2 = "SELECT substring(timestamp, 0, 10) AS sale_date, SUM(price) AS total_sales \
          FROM purchases GROUP BY substring(timestamp, 0, 10) \
          ORDER BY total_sales DESC LIMIT 1"
results = sqlContext.sql(query2).map(lambda l: l.sale_date)
for result in results.collect():
    print result

<type 'unicode'>
2012-03-17


In [134]:
# problem 2 - query 3
query3 = "SELECT MIN(price) AS min_val FROM purchases WHERE category='Computers'"
results = sqlContext.sql(query3).map(lambda l: l.min_val)
for result in results.collect():
    print result


0.379999995232


In [110]:
# problem 2 - query 4
query4 = "SELECT COUNT(DISTINCT category) AS dist_cat_num FROM purchases"
results = sqlContext.sql(query4).map(lambda l: l.dist_cat_num)
for result in results.collect():
    print result

18


In [160]:
# problem 2 - query 5
query5 = "SELECT location, SUM(price) AS store_sales FROM purchases GROUP BY location ORDER BY store_sales LIMIT 1"
results = sqlContext.sql(query5).map(lambda l: l.location)
for result in results.collect():
    print result

Plano
